<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

*   Understand the dataset for Chicago Public School level performance
*   Store the dataset in an Db2 database on IBM Cloud instance
*   Retrieve metadata about tables and columns and query data from mixed case columns
*   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01&download=true&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**:

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01">link</a>.

**NOTE**:

For the learners who are encountering issues with loading from .csv in DB2 on Firefox, you can download the .txt files and load the data with those: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.txt?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR.

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next".

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database

The following modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [1]:
%load_ext sql

In [2]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://kmx22828:p1uKv0i9qXYnCbMt@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB?security=SSL

'Connected: kmx22828@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [3]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select *  from syscat.tables where TABSCHEMA = 'KMX22828';

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
KMX22828,TEST,KMX22828,U,T,N,None,None,None,None,2022-09-19 23:14:17.753320,2022-09-19 23:14:17.753320,2022-09-19 23:14:17.753320,2022-09-19 23:59:54.131941,2,4,5447,0,0,0,1,-1,-1,-1,0,KMX22828SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,0,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2022-09-19 23:14:17.753320,0,,None,None,N,KMX22828,,,,0001-01-01,,N,R,N,-1.0,None
KMX22828,PETSALE,KMX22828,U,T,N,None,None,None,None,2022-09-19 23:18:51.627787,2022-09-19 23:44:31.009786,2022-09-19 23:44:31.009792,2022-10-12 22:00:02.563535,5,5,5447,5,1,0,1,-1,-1,-1,0,KMX22828SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,54,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2022-09-19 23:18:51.627787,0,,None,None,N,KMX22828,,,,2022-09-19,,N,R,N,-1.0,None
KMX22828,PETRESCUE,KMX22828,U,T,N,None,None,None,None,2022-09-30 18:20:19.196084,2022-09-30 18:20:19.358642,2022-09-30 18:20:19.358648,2022-09-30 18:24:05.505994,5,11,5447,9,1,0,1,-1,-1,-1,0,KMX22828SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,38,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2022-09-30 18:20:19.196084,0,,None,None,N,KMX22828,,,,2022-09-30,,N,R,N,-1.0,None
KMX22828,EMPLOYEES,KMX22828,U,T,N,None,None,None,None,2022-09-30 19:11:45.176294,2022-09-30 19:11:45.285483,2022-09-30 19:11:45.285489,2022-09-30 19:19:05.885559,11,6,5447,10,1,0,2,-1,-1,-1,0,KMX22828SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,118,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2022-09-30 19:11:45.176294,0,,None,None,N,KMX22828,,,,0001-01-01,,N,R,N,-1.0,None
KMX22828,JOB_HISTORY,KMX22828,U,T,N,None,None,None,None,2022-09-30 19:11:45.301309,2022-09-30 19:11:45.424850,2022-09-30 19:11:45.424856,2022-09-30 20:24:56.205363,4,7,5447,10,1,0,2,-1,-1,-1,0,KMX22828SPACE1,None,None,0,0,0,2,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,43,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2022-09-30 19:11:45.301309,0,,None,None,N,KMX22828,,,,2022-09-30,,N,R,N,-1.0,None
KMX22828,JOBS,KMX22828,U,T,N,None,None,None,None,2022-09-30 19:11:45.448807,2022-09-30 19:11:45.563812,2022-09-30 19:11:45.563818,2022-09-30 19:39:13.768927,4,8,5447,10,1,0,2,-1,-1,-1,0,KMX22828SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,53,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2022-09-30 19:11:45.448807,0,,None,None,N,KMX22828,,,,2022-09-30,,N,R,N,-1.0,None
KMX22828,DEPARTMENTS,KMX22828,U,T,N,None,None,None,None,2022-09-30 19:11:45.591038,2022-09-30 19:11:45.698733,2022-09-30 19:11:45.698740,2022-09-30 20:06:23.800484,4,9,5447,3,1,0,2,-1,-1,-1,0,KMX22828SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F

Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [4]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from syscat.columns where tabname = 'CHICAGO_PUBLIC_SCHOOLS';

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


1
78


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [5]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select colname, colno, typename, length from syscat.columns where tabname = 'CHICAGO_PUBLIC_SCHOOLS';

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


colname,colno,typename,length
SCHOOL_ID,0,INTEGER,4
NAME_OF_SCHOOL,1,VARCHAR,64
"Elementary, Middle, or High School",2,VARCHAR,2
STREET_ADDRESS,3,VARCHAR,29
CITY,4,VARCHAR,7
STATE,5,VARCHAR,2
ZIP_CODE,6,INTEGER,4
PHONE_NUMBER,7,VARCHAR,14
LINK,8,VARCHAR,78
NETWORK_MANAGER,9,VARCHAR,40


Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [6]:
%sql select count(*) from CHICAGO_PUBLIC_SCHOOLS where "Elementary, Middle, or High School" = 'ES';

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


1
462


Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [7]:
%sql select MAX(SAFETY_SCORE) as MAX_SAFETY_SCORE from CHICAGO_PUBLIC_SCHOOLS;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


max_safety_score
99


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [8]:
%%sql 
select NAME_OF_SCHOOL, SAFETY_SCORE from CHICAGO_PUBLIC_SCHOOLS 
where SAFETY_SCORE = (select MAX(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS) 


 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


name_of_school,safety_score
Abraham Lincoln Elementary School,99
Alexander Graham Bell Elementary School,99
Annie Keller Elementary Gifted Magnet School,99
Augustus H Burley Elementary School,99
Edgar Allan Poe Elementary Classical School,99
Edgebrook Elementary School,99
Ellen Mitchell Elementary School,99
James E McDade Elementary Classical School,99
James G Blaine Elementary School,99
LaSalle Elementary Language Academy,99


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [9]:
%%sql 
select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE from CHICAGO_PUBLIC_SCHOOLS
order by AVERAGE_STUDENT_ATTENDANCE desc nulls last limit 10;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


name_of_school,average_student_attendance
John Charles Haines Elementary School,98.40%
James Ward Elementary School,97.80%
Edgar Allan Poe Elementary Classical School,97.60%
Orozco Fine Arts & Sciences Elementary School,97.60%
Rachel Carson Elementary School,97.60%
Annie Keller Elementary Gifted Magnet School,97.50%
Andrew Jackson Elementary Language Academy,97.40%
Lenart Elementary Regional Gifted Center,97.40%
Disney II Magnet School,97.30%
John H Vanderpoel Elementary Magnet School,97.20%


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [10]:
%%sql 
select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE from CHICAGO_PUBLIC_SCHOOLS
order by AVERAGE_STUDENT_ATTENDANCE limit 5;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [11]:
%%sql 
SELECT NAME_OF_SCHOOL, REPLACE(AVERAGE_STUDENT_ATTENDANCE,'%','')
FROM CHICAGO_PUBLIC_SCHOOLS LIMIT 5

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


name_of_school,2
Abraham Lincoln Elementary School,96.00
Adam Clayton Powell Paideia Community Academy Elementary School,95.60
Adlai E Stevenson Elementary School,95.70
Agustin Lara Elementary Academy,95.50
Air Force Academy High School,93.30


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [12]:
%%sql 
select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE
from CHICAGO_PUBLIC_SCHOOLS
where DECIMAL(REPLACE(AVERAGE_STUDENT_ATTENDANCE,'%','')) < 70
order by AVERAGE_STUDENT_ATTENDANCE;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%
Manley Career Academy High School,66.80%
Chicago Vocational Career Academy High School,68.80%
Roberto Clemente Community Academy High School,69.60%


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [17]:
%%sql 
select SUM(COLLEGE_ENROLLMENT) AS total_enrollment, COMMUNITY_AREA_NAME from CHICAGO_PUBLIC_SCHOOLS GROUP BY COMMUNITY_AREA_NAME;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


total_enrollment,community_area_name
6864,ALBANY PARK
4823,ARCHER HEIGHTS
1458,ARMOUR SQUARE
6483,ASHBURN
4175,AUBURN GRESHAM
10933,AUSTIN
1522,AVALON PARK
3640,AVONDALE
14386,BELMONT CRAGIN
1636,BEVERLY


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [18]:
%%sql 
select SUM(COLLEGE_ENROLLMENT) AS total_enrollment, COMMUNITY_AREA_NAME 
from CHICAGO_PUBLIC_SCHOOLS 
GROUP BY COMMUNITY_AREA_NAME
order by total_enrollment limit 5;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


total_enrollment,community_area_name
140,OAKLAND
531,FULLER PARK
549,BURNSIDE
786,OHARE
871,LOOP


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [21]:
%%sql 
select SAFETY_SCORE, NAME_OF_SCHOOL
from CHICAGO_PUBLIC_SCHOOLS 
order by SAFETY_SCORE limit 5;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


safety_score,name_of_school
1,Edmond Burke Elementary School
5,Luke O'Toole Elementary School
6,George W Tilton Elementary School
11,Foster Park Elementary School
13,Emil G Hirsch Metropolitan High School


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score \
FROM schools \
ORDER BY safety_score \
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [34]:
%%sql 
select hardship_index 
   from CHICAGO_SOCIOECONOMIC_DATA CD, CHICAGO_PUBLIC_SCHOOLS CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


hardship_index
6.0


In [27]:
%sql select colname, colno, typename, length from syscat.columns where tabname = 'CHICAGO_SOCIOECONOMIC_DATA';

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


colname,colno,typename,length
index,0,BIGINT,8
CA,1,DOUBLE,8
COMMUNITY_AREA_NAME,2,CLOB,1048576
PERCENT_OF_HOUSING_CROWDED,3,DOUBLE,8
PERCENT_HOUSEHOLDS_BELOW_POVERTY,4,DOUBLE,8
PERCENT_AGED_16_UNEMPLOYED,5,DOUBLE,8
PERCENT_AGED_25_WITHOUT_HIGH_SCHOOL_DIPLOMA,6,DOUBLE,8
PERCENT_AGED_UNDER_18_OR_OVER_64,7,DOUBLE,8
PER_CAPITA_INCOME_,8,BIGINT,8
HARDSHIP_INDEX,9,DOUBLE,8


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [43]:
%%sql 
SELECT CD.HARDSHIP_INDEX, CPS.COMMUNITY_AREA_NAME, CPS.COLLEGE_ENROLLMENT 
FROM CHICAGO_PUBLIC_SCHOOLS CPS, CHICAGO_SOCIOECONOMIC_DATA CD
WHERE CD.CA = CPS.COMMUNITY_AREA_NUMBER 
ORDER BY COLLEGE_ENROLLMENT DESC LIMIT 1;

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


hardship_index,community_area_name,college_enrollment
6.0,NORTH CENTER,4368


In [44]:
%%sql 
select ca, community_area_name, hardship_index 
from chicago_socioeconomic_data
where ca in (select community_area_number from CHICAGO_PUBLIC_SCHOOLS order by college_enrollment desc limit 1)

 * ibm_db_sa://kmx22828:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/BLUDB
Done.


ca,community_area_name,hardship_index
5.0,North Center,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2022-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                        |
| ----------------- | ------- | ----------------- | ----------------------------------------- |
| 2021-07-09        | 2.4     | Malika            | Updated connection string                 |
| 2021-05-19        | 2.3     | Lakshmi Holla     | Updated question                          |
| 2021-04-20        | 2.2     | Malika            | Added the libraries                       |
| 2020-11-27        | 2.1     | Sannareddy Ramesh | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya           | Moved lab to course repo in GitLab        |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
